In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from imgaug import augmenters as iaa
import torch

torch.backends.cudnn.benchmark = True

from dataloader import get_test_dataloader
from preprocess import transformer, img_transform, reverse_img_transform
from models import RetinaNet
from train import load_components
from inference import InferenceTransform
from utils import Visualiser
import pickle
import gc
import json
from evaluation import support_evaluate_model
gc.collect(2)

/opt/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


0

In [3]:
configs_dir = "configs/final/subset0/0/config.json"

In [4]:
parameters = json.load(open(configs_dir,'r'))

## 1. Parameters

In [5]:
hyper_params = parameters["hyperparams"]
dir_params = parameters["dirparams"]
project_params = parameters["projectconfig"]

project_name = project_params["project_name"]
experiment_name = project_params["experiment_name"]

## 2. Initialisations

In [6]:
clsids_to_names = json.load(open(dir_params["clsids_to_names_dir"],'r'))
clsids_to_idx = json.load(open(dir_params["clsids_to_idx_dir"],'r'))
idx_to_cls_ids = {v: k for k, v in clsids_to_idx.items()}
idx_to_names = {k: clsids_to_names[v] for k, v in idx_to_cls_ids.items()}

## 3. Dataloaders

In [7]:
test_seq = iaa.Sequential([
        iaa.Resize({"height": hyper_params["img_dim"], "width": hyper_params["img_dim"]})
    ])
test_transform_fn = transformer(test_seq, img_transform)
test_dl = get_test_dataloader(
        "dataset/test/test", 
        "data_info/test/test-idx_to_id.pkl", 
        test_transform_fn, 
        hyper_params["bs"], 
        hyper_params["num_workers"]
    )

## 4. Model

In [8]:
retinanet = RetinaNet(
        hyper_params["backbone"], 
        hyper_params["num_classes"],
        hyper_params["ratios"], 
        hyper_params["scales"], 
        device=hyper_params["device"], 
        pretrained = hyper_params["pretrained"], 
        freeze_bn = hyper_params["freeze_bn"],
        prior=0.01, 
        feature_size=256, 
        pyramid_levels = [3, 4, 5, 6, 7],
        criterion=None
    )
retinanet = retinanet.to(hyper_params["device"])

## 5. Visualisations

In [9]:
vis = Visualiser(
        hyper_params["num_classes"],
        idx_to_names,
        reverse_img_transform
    )

## 6. Inference

In [10]:
inferencer = InferenceTransform(
        idx_to_names,
        idx_to_cls_ids,
        hyper_params["regress_factor"]
    ) 

In [11]:
load_components(retinanet, None, None, hyper_params["save_dir"]+"/final.pth")

Loading from checkpoint: configs/final/subset0/0/final.pth


In [12]:
if torch.cuda.device_count() > 1:
    print("Using Multiple GPUs")
    retinanet = torch.nn.DataParallel(retinanet, device_ids=range(torch.cuda.device_count())) 
retinanet = retinanet.to(hyper_params["device"])

In [17]:
eval_params = {
    "overlap":hyper_params["overlap"],
    "cls_thresh": hyper_params["cls_thresh"]
}

## 7. Prepare Evaluation

In [ ]:
list_results_dict = defaultdict(list)

In [19]:
resultant_dict = support_evaluate_model(retinanet, 
                      test_dl, 
                       inferencer, 
                       vis, 
                       cls_thresh=eval_params["cls_thresh"], 
                       overlap=eval_params["overlap"], 
                       device=hyper_params["device"], 
                       save_dir="test", 
                       display = False, 
                       create_result=True,
                       list_results_dict = list_results_dict)

Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/opt/anaconda3/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
Traceback (most recen

KeyboardInterrupt: 